In [61]:
import pandas as pd
import imdb

In [70]:
df = pd.read_csv("./data/df_final.csv")
df

,year_film,year_ceremony,ceremony,film,winner,original_language,original_title,release_date,vote_average,genre,rating,duration
0,1927,1928,1,The Racket,False,en,The Racket,1928-11-01,6.400,"['Crime', 'Drama']",6.6,84.0
1,1927,1928,1,7th Heaven,False,en,7th Heaven,1927-09-10,7.317,"['Drama', 'Romance']",7.6,110.0
2,1927,1928,1,Wings,True,en,Wings,1927-08-12,7.198,"['Drama', 'Action', 'War', 'Romance']",7.5,144.0
3,1928,1929,2,Alibi,False,en,Alibi,1929-04-20,5.656,"['Action', 'Crime', 'Thriller']",5.7,91.0
4,1928,1929,2,In Old Arizona,False,en,In Old Arizona,1928-12-25,5.208,"['Action', 'Western']",5.5,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...
586,2022,2023,95,The Fabelmans,False,en,The Fabelmans,2022-11-11,7.692,"['Drama', 'Comedy']",7.5,151.0
587,2022,2023,95,Tár,False,en,TÁR,2022-09-23,7.143,"['Drama', 'Music']",7.4,158.0
588,2022,2023,95,Top Gun: Maverick,False,en,Top Gun: Maverick,2022-05-24,8.237,"['Action', 'Drama']",8.3,130.0
589,2022,2023,95,Triangle of Sadness,False,en,Triangle of Sadness,2022-09-18,7.116,"['Comedy', 'Drama']",7.3,147.0


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year_film          591 non-null    int64  
 1   year_ceremony      591 non-null    int64  
 2   ceremony           591 non-null    int64  
 3   film               591 non-null    object 
 4   winner             591 non-null    bool   
 5   original_language  591 non-null    object 
 6   original_title     591 non-null    object 
 7   release_date       591 non-null    object 
 8   vote_average       591 non-null    float64
 9   genre              591 non-null    object 
 10  rating             581 non-null    float64
 11  duration           582 non-null    float64
dtypes: bool(1), float64(3), int64(3), object(5)
memory usage: 51.5+ KB


### Ratings

In [72]:
ratings_nulos = df[df["rating"].isna()]
ratings_nulos

,year_film,year_ceremony,ceremony,film,winner,original_language,original_title,release_date,vote_average,genre,rating,duration
7,1928,1929,2,The Patriot,False,en,The Patriot,1928-09-01,5.000,['Drama'],NaN,113.0
85,1938,1939,11,Grand Illusion,False,fr,La Grande Illusion,1937-06-04,7.858,"['Drama', 'History', 'War']",NaN,NaN
151,1946,1947,19,Henry V,False,en,The Chronicle History of King Henry the Fifth ...,1944-11-24,6.604,"['Drama', 'History', 'War']",NaN,NaN
174,1950,1951,23,Sunset Boulevard,False,en,Sunset Boulevard,1950-08-17,8.322,['Drama'],NaN,NaN
200,1956,1957,29,Around the World in Eighty Days,True,en,Around the World in Eighty Days,1956-10-17,6.664,"['Adventure', 'Comedy', 'Family', 'Fantasy', '...",NaN,NaN
252,1966,1967,39,The Russians Are Coming! The Russians Are Coming!,False,en,The Russians Are Coming! The Russians Are Coming!,1966-05-25,6.300,"['Comedy', 'War']",NaN,NaN
286,1973,1974,46,Cries and Whispers,False,sv,Viskningar och rop,1972-12-21,7.939,['Drama'],NaN,NaN
308,1977,1978,50,Star Wars,False,en,Star Wars,1977-05-25,8.205,"['Adventure', 'Action', 'Science Fiction']",NaN,NaN
369,1989,1990,62,My Left Foot: The Story of Christy Brown,False,en,My Left Foot: The Story of Christy Brown,1989-04-07,7.521,['Drama'],NaN,NaN
561,2019,2020,92,Once Upon a Time… in Hollywood,False,en,Once Upon a Time… in Hollywood,2019-07-24,7.439,"['Comedy', 'Drama', 'Thriller']",NaN,NaN


In [75]:
ia = imdb.IMDb()
for index, row in df.iterrows():
    if pd.isna(row["rating"]):
        film = row["film"]
        year = row["year_film"]

        movie_search = ia.search_movie(film, year)

        matching_movie = None
        for result in movie_search:
            # IMDb movies may not have a 'year' attribute, so we need to check its existence
            if "year" in result and abs(result["year"] - year) <= 2:
                matching_movie = result
                break
        if matching_movie:
            movie_id = matching_movie.movieID
            movie = ia.get_movie(movie_id)

            # Access the rating information
            rating = movie.data.get('rating', None)

            # Update the DataFrame with the retrieved rating
            df.at[index, 'rating'] = rating
        

In [76]:
ratings_nulos

,year_film,year_ceremony,ceremony,film,winner,original_language,original_title,release_date,vote_average,genre,rating,duration
7,1928,1929,2,The Patriot,False,en,The Patriot,1928-09-01,5.000,['Drama'],NaN,113.0
85,1938,1939,11,Grand Illusion,False,fr,La Grande Illusion,1937-06-04,7.858,"['Drama', 'History', 'War']",NaN,NaN
151,1946,1947,19,Henry V,False,en,The Chronicle History of King Henry the Fifth ...,1944-11-24,6.604,"['Drama', 'History', 'War']",NaN,NaN
174,1950,1951,23,Sunset Boulevard,False,en,Sunset Boulevard,1950-08-17,8.322,['Drama'],NaN,NaN
200,1956,1957,29,Around the World in Eighty Days,True,en,Around the World in Eighty Days,1956-10-17,6.664,"['Adventure', 'Comedy', 'Family', 'Fantasy', '...",NaN,NaN
252,1966,1967,39,The Russians Are Coming! The Russians Are Coming!,False,en,The Russians Are Coming! The Russians Are Coming!,1966-05-25,6.300,"['Comedy', 'War']",NaN,NaN
286,1973,1974,46,Cries and Whispers,False,sv,Viskningar och rop,1972-12-21,7.939,['Drama'],NaN,NaN
308,1977,1978,50,Star Wars,False,en,Star Wars,1977-05-25,8.205,"['Adventure', 'Action', 'Science Fiction']",NaN,NaN
369,1989,1990,62,My Left Foot: The Story of Christy Brown,False,en,My Left Foot: The Story of Christy Brown,1989-04-07,7.521,['Drama'],NaN,NaN
561,2019,2020,92,Once Upon a Time… in Hollywood,False,en,Once Upon a Time… in Hollywood,2019-07-24,7.439,"['Comedy', 'Drama', 'Thriller']",NaN,NaN


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year_film          591 non-null    int64  
 1   year_ceremony      591 non-null    int64  
 2   ceremony           591 non-null    int64  
 3   film               591 non-null    object 
 4   winner             591 non-null    bool   
 5   original_language  591 non-null    object 
 6   original_title     591 non-null    object 
 7   release_date       591 non-null    object 
 8   vote_average       591 non-null    float64
 9   genre              591 non-null    object 
 10  rating             590 non-null    float64
 11  duration           582 non-null    float64
dtypes: bool(1), float64(3), int64(3), object(5)
memory usage: 51.5+ KB


In [78]:
ia = imdb.IMDb()
for index, row in df.iterrows():
    if pd.isna(row["duration"]):
        film = row["film"]
        year = row["year_film"]

        movie_search = ia.search_movie(film, year)

        matching_movie = None
        for result in movie_search:
            if "year" in result and abs(result["year"] - year) <= 2:
                matching_movie = result
                break
        if matching_movie:
            movie_id = matching_movie.movieID
            movie = ia.get_movie(movie_id)

            duration = movie.data.get('runtimes', None)

            df.at[index, 'duration'] = duration
        

C:\Users\Nur\AppData\Local\Temp\ipykernel_25792\553916578.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '113' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'duration'] = duration


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year_film          591 non-null    int64  
 1   year_ceremony      591 non-null    int64  
 2   ceremony           591 non-null    int64  
 3   film               591 non-null    object 
 4   winner             591 non-null    bool   
 5   original_language  591 non-null    object 
 6   original_title     591 non-null    object 
 7   release_date       591 non-null    object 
 8   vote_average       591 non-null    float64
 9   genre              591 non-null    object 
 10  rating             590 non-null    float64
 11  duration           591 non-null    object 
dtypes: bool(1), float64(2), int64(3), object(6)
memory usage: 51.5+ KB


In [81]:
df.to_csv("./data/final_final.csv", index=False)